# AI-ассистент по моему GitHub портфолио

**Цель:** Разработать MVP (прототип) чат-бота, который отвечает на вопросы о моем портфолио на GitHub, анализируя как общую, так и детальную информацию по проектам.

**Задачи:**
1. Подготовка и выбор стека
2. Сбор и индексация данных
3. Создание RAG-цепочки
4. Подключение интерфейса и тестирование

## Подготовка и выбор стека

**Определение MVP:** MVP - это Python скрипт app.py, содержащий функцию `query_portfolio`, которая принимает на вход вопрос в виде текста и возвращает текстовый ответ, сгенерированный на основе текстового содержимого GitHub-портфолио.

**Стек технологий:**
- Оркестратор: LangChain - наиболее мощный и гибкий инструмент;
- Векторная БД: ChromaDB - удобна для прототипирования;
- Embeddings модель: all-MiniLM-L6-v2 - популярная, легкая и быстрая с хорошим балансом скорость/качество;
- LLM: Google Gemini 2.5 Flash через API;
- Веб-интерфейс: Gradio - для быстрого прототипирования.

In [ ]:
!pip install -q google-ai-generativelanguage==0.6.15 \
langchain langchain-community langchain-huggingface \
langchain-google-genai langchain-unstructured langchain-chroma \
unstructured sentence-transformers jq gradio chromadb markdown

In [21]:
# Импорт библиотек
import os
import logging
import requests

import gradio as gr
from tqdm import tqdm
from langchain_community.document_loaders import (
    UnstructuredMarkdownLoader, NotebookLoader,
    TextLoader, PythonLoader, JSONLoader,

)
from langchain_unstructured.document_loaders import UnstructuredLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# Основные настройки
OWNER = "rzarubayev"
REPO = "machine-learning-portfolio"
BRANCH = "main"

# Исходные данные
DATA_PATH = "data"
# Путь к векторной базе
CHROMA_DIR = "chroma"
# Модель для эмбеддингов и LLM
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "gemini-2.5-flash"

# Используемые загрузчики и их параметры
FILE_LOADERS = {
    ".ipynb": NotebookLoader,
    ".md": UnstructuredMarkdownLoader,
    ".py": PythonLoader,
    ".txt": TextLoader,
    ".sh": TextLoader,
    ".yaml": UnstructuredLoader,
    ".yml": UnstructuredLoader,
    ".json": JSONLoader
}
LOADER_PARAMS = {
    "NotebookLoader": {
        "include_outputs": False, # Не используем вывод ячеек
        "remove_newline": True    # Убираем лишние переносы строк
    },
    "UnstructuredLoader": {
        "mode": "single"
    },
    "TextLoader": {
        "encoding": "utf8"
    },
    "JSONLoader": {
        "jq_schema": ".",
        "text_content": False
    },
}

# Установка уровня логирования в unstructured
logging.getLogger("unstructured").setLevel(logging.ERROR)

**Настройка доступа к Gemini API:**
Для получения доступа к API LLM Gemini 2.5 Flash необходимо сделать следующее:
1. Зайти на сайт [Google AI studio](https://aistudio.google.com/apikey);
2. Создать API ключ;
3. Добавить ключ в переменную окружения GOOGLE_API_KEY (секрет в Hugging Face Spaces)

In [23]:
# Проверка загрузки модели
llm = ChatGoogleGenerativeAI(model=LLM_MODEL)
llm.model

'models/gemini-2.5-flash'

## Сбор и индексация данных

Подготовим функции для загрузки списка файлов из GitHub, имеющих соответствующее расширение.

In [ ]:
# Фукнция для получения списка файлов из
def download_repo_files(owner: str, repo: str, branch: str, file_types: tuple,
                        output_dir: str):
  """
  Получает список всех файлов во всех папках репозитория с помощью GitHub API,
  загружает файлы в указанную директорию.
  """
  api_url = f"https://api.github.com/repos/{owner}/{repo}/git/trees/{branch}?recursive=1"
  print(f"Запрашиваем структуру репозитория {api_url}")
  try:
    # Получаем список файлов
    response = requests.get(api_url)
    response.raise_for_status()
    data = response.json()
    if data.get("truncated"):
      print("Внимание: список файлов был усечен, так как репозиторий слишком большой")
    file_list = data.get("tree", [])
  except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе к GitHub API: {e}")
    return
  files = [
      f for f in file_list
      if f["type"] == "blob" and f["path"].endswith(file_types)
  ]
  if not files:
    print(f"В репозитории не найдены файлы с раширениями {file_types}")
    return

  for f in tqdm(files, desc="Скачивание файлов"):
    # Получаем URL для доступа к самим файлам
    file_path = f["path"]
    raw_url = f"https://raw.githubusercontent.com/{owner}/{repo}/{branch}/{file_path}"
    # Создаем папки, если они отсутствуют
    local_file_path = os.path.join(output_dir, file_path)
    os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
    # Скачиваем файлы
    try:
      response = requests.get(raw_url)
      response.raise_for_status()
      with open(local_file_path, "w", encoding="utf8") as file:
        file.write(response.text)
    except requests.exceptions.RequestException as e:
      print(f"Ошибка при скачивании файла {file_path}: {e}")
      tqdm.write(f" - Ошибка при скачивании файла {file_path}: {e}")

Загрузим файлы в папку.

In [25]:
if not os.path.exists(DATA_PATH):
  download_repo_files(OWNER, REPO, BRANCH, tuple(FILE_LOADERS.keys()), DATA_PATH)

Подготовим функцию для загрузки файлов, которая сформирует базу знаний для LLM (список документов LangChain).

In [26]:
def load_docs(data_path: str, loaders_map: dict,
              params_map: dict) -> tuple[list, list]:
  """
  Загружает все файлы посредством соответствующих загрузчиков langchain,
  используя словари для выбора нужного загрузчика и его параметров.
  """
  # Инициализируем переменные
  general_docs, project_docs, target_files = [], [], []
  supported_ext = tuple(loaders_map.keys())

  # Получим только известные файлы (на всякий случай, если там что-то уже было)
  for root, _, files in os.walk(data_path):
    for f in files:
      if f.endswith(supported_ext):
        target_files.append(os.path.join(root,f))

  if not target_files:
    print(f"В папке {data_path} не найдены файлы с расширением {supported_ext}")
    return [], []

  # Обработка файлов
  print(f"Найдено {len(target_files)} файлов. Начинаем обработку...")
  for file_path in tqdm(target_files, desc="Обработка файлов"):
    try:
      # Получение загрузчика по расширению
      file_ext = os.path.splitext(file_path)[1]
      loader_class = loaders_map[file_ext]

      if not loader_class:
        tqdm.write(
          f" - пропуск файла: не найден загрузчик для расширения {file_ext}")
        continue

      loader_params = params_map.get(loader_class.__name__, {})
      loader = loader_class(file_path, **loader_params)
      docs = loader.load()
      # Объединение в один документ
      content = "\n\n".join([doc.page_content for doc in docs])
      doc = Document(page_content=content)
      # Добавление метаданных (источник, категория, проект)
      # в соответствии со структурой портфолио
      rel_path = os.path.relpath(file_path, data_path)
      doc.metadata["source"] = rel_path

      parts = rel_path.split(os.sep)

      if len(parts) == 1:
        # Файлы из корневой директории
        doc.metadata["category"] = "general"
        general_docs.append(doc)
      else:
        # Получаем категорию по названию директории
        doc.metadata["category"] = parts[0]
        if len(parts) == 2:
          # Проект - имя файла, если нет субдиректорий
          doc.metadata["project"] = os.path.splitext(parts[1])[0]
        else:
          # Если это субдиректория, проект - ее имя
          doc.metadata["project"] = parts[1]
        project_docs.append(doc)

    except Exception as e:
      tqdm.write(f" - Ошибка при обработке файла {file_path}: {e}")

  return general_docs, project_docs

Функция готова, получим список документов LangChain.

In [27]:
general, projects = load_docs(DATA_PATH, FILE_LOADERS, LOADER_PARAMS)
sample_doc = projects[7]
print()
print(f"Количество общих документов: {len(general)}")
print(f"Количество проектных документов: {len(projects)}")
print(f"Пример документа: \n{sample_doc.page_content[:200]}")

Найдено 125 файлов. Начинаем обработку...


Обработка файлов: 100%|██████████| 125/125 [00:12<00:00,  9.81it/s]


Количество общих документов: 2
Количество проектных документов: 123
Пример документа: 
{"id": 116938, "rooms": 1, "total_area": 32.5999984741, "kitchen_area": 6, "living_area": 18.7000007629, "floor": 9, "studio": false, "is_apartment": false, "building_type_int": 4, "build_year": 1970,


Документы загружены, для подачи их в LLM в качестве контекста, необходимо нарезать их на более мелкие, семантически связанные части — чанки. Это важный шаг, так как поиск по небольшим чанкам работает гораздо точнее, а также у языковых моделей есть ограничение на размер входного контекста. Мы будем использовать RecursiveCharacterTextSplitter, который интеллектуально разбивает текст, с двумя ключевыми параметрами: chunk_size=1000 — оптимальный размер одного чанка в символах для сохранения контекста, и chunk_overlap=200 — пересечение между чанками, чтобы не терять смысл на их стыках.

Применим гибридный подход к формированию контекста: общие документы будем подавать в модель целиком для сохранения глобальной информации, а проектные файлы нарежем на чанки. Это позволит ретриверу находить наиболее релевантные фрагменты для ответа на конкретные вопросы.

In [28]:
if projects:
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=200
  )
  chunks = text_splitter.split_documents(projects)
  print(f"Количество чанков: {len(chunks)}")
  print(f"Пример чанка: \n{chunks[37].page_content[:200]}")

Количество чанков: 2994
Пример чанка: 
./services/stop_compose.sh

Либо можно остановить его вручную из папки services\:

cd services/
docker compose stop

4. Скрипт симуляции нагрузки

Скрипт генерирует 600 запросов в течение ~150 секунд.


Следующий шаг — создание векторной базы знаний. Мы инициализируем модель эмбеддингов, которая преобразует каждый текстовый чанк в числовой вектор и сохраним на диск векторное хранилище из этих чанков с помощью ChromaDB. Этот процесс называется индексацией и позволит выполнять быстрый семантический поиск по нашей базе знаний.

In [29]:
if chunks:
  embedding = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
  if os.path.exists(CHROMA_DIR):
    # Загружаем базу, если она существует
    vector_store = Chroma(
        persist_directory=CHROMA_DIR,
        embedding_function=embedding
    )
  else:
    # Создаем базу, если ее нет
    vector_store = Chroma.from_documents(
        documents=chunks,
        embedding=embedding,
        persist_directory=CHROMA_DIR
    )
  print(f"Количество векторов: {vector_store._collection.count()}")


Количество векторов: 2994


Наша база знаний создана и векторизована, теперь нам нужен инструмент, который сможет эффективно извлекать из нее релевантную информацию. В LangChain эту роль выполняет Ретривер (Retriever). Мы создадим его из нашего векторного хранилища и настроим так, чтобы по любому запросу он находил 15 наиболее семантически близких документа (k=15), которые затем будут переданы в качестве контекста для LLM.

In [30]:
if vector_store:
  retriever = vector_store.as_retriever(
      search_type="similarity",
      search_kwargs={"k": 15}
  )

  test_query = "Какой имеется опыт с multi-label классификацией"
  rel_docs = retriever.invoke(test_query)
  print(f"Количество найденных документов: {len(rel_docs)}")
  print(f"Пример документа: \n{rel_docs[0].page_content[:200]}")

Количество найденных документов: 15
Пример документа: 
В целом поставленную задачу можно интерпретировать как задачу многометочной (multi-label) классификации, в том числе возможно разернуть продукты по клиентам а также дополнить признаками по продукту, н


## Создание RAG цепочки

Чтобы сделать нашего ассистента умнее, реализуем более сложную, трехэтапную архитектуру. Вместо того чтобы слепо доверять результатам векторного поиска, сначала заставим модель подумать. На первом этапе модель суммаризует историю переписки оставив максимально релевантный контент к вопросу пользователя. На втором этапе модель будет работать как исследователь, который определит наиболее релевантные файлы проектов. И только последняя модель "ассистент" будет формировать финальный ответ на основе всего этого расширенного контекста.

Начнем с создания "мозга" для суммаризации истории переписки — подготовим для него специальный промпт.

In [31]:
HISTORY_PROMPT_TEMPLATE = """
Твоя задача — проанализировать историю диалога и новый вопрос пользователя,
а затем создать краткую и самодостаточную сводку.
Эта сводка будет использована для подготовки ответа другой AI модели.

Внимательно изучи историю и вопрос. Затем сделай ОДНО из двух:
1. **Переформулируй вопрос:** если новый вопрос пользователя зависит от
контекста (например, содержит ссылку на предыдущий вопрос или ответ,
со словами: "это", "там", "а в нем" и аналогичные), перепиши его так,
чтобы он был понятен без истории чата как контекст для модели.
2. **Создай резюме:** если переформулировать вопрос сложно,
напиши очень короткое (2-3 предложения) ключевой информации из истории,
необходимой для ответа на новый вопрос.
В конце добавь новый вопрос пользователя.

История переписки:
{chat_history}
Новый вопрос:
{question}
Сжатый контекст:
"""

history_prompt = PromptTemplate(
    template=HISTORY_PROMPT_TEMPLATE,
    input_variables=["chat_history", "question"]
)

Подготовим промп для "исследователя". Для него подаем общую информацию, вопрос пользователя, найденные ретривером релевантные чанки и полный список всех файлов в нашей базе знаний. Ее задача — не отвечать на вопрос, а проанализировать все вводные данные и решить, какие документы нужно прочитать целиком для наиболее полного ответа.

In [32]:
RESEARCH_PROMPT_TEMPLATE = """
Ты умный AI-исследователь. Твоя задача - помочь ответить на вопрос пользователя.
Ниже представлены общие сведения по проектам и владельце портфолио, сам вопрос,
несколько релевантных фрагментов, найденных в базе знаний и полный список всех
доступных файлов проектов.

Проанализируй эту информацию и реши, какие файлы из полного списка нужно
прочитать ЦЕЛИКОМ, чтобы дать наиболее полный и точный ответ.
Учитывай и те файлы, из которых взяты фаргменты, и любые другие,
которые кажутся тебе релевантными. Не добавляй файлы, которые не считаешь
релевантными. Составь список по уменьшению релевантности.

Твой ответ должен быть ТОЛЬКО списком путей к файлам в формате чистого JSON.
Никаких лишних слов и кавычек.
Если не найдешь ничего подходящего, верни пустой список - []

--- Общая информация о проектах и владельце портфолио ---
{general_context}

--- Вопрос пользователя ---
{question}

--- Найденные фрагменты (подсказки) ---
{retrieved_chunks}

--- Список всех доступных файлов ---
{all_files}

--- Ответ в формате JSON ---
"""

research_prompt = PromptTemplate(
    template=RESEARCH_PROMPT_TEMPLATE,
    input_variables=[
        "general_context", "question",
        "retrieved_chunks", "all_files"
    ]
)

Теперь, когда у нас есть промпт "исследователя", по которому модель способна определить наиболее релевантные файлы, подготовим основной промпт для нашей модели. Эта модель будет играть роль AI-ассистента. В ее задачу входит синтезировать исчерпывающий и связный ответ для пользователя, опираясь на два источника: постоянный общий контекст и динамический контекст, состоящий из полного содержимого файлов, которые на предыдущем шаге порекомендовала "исследовательская" модель.

In [33]:
ANSWER_PROMPT_TEMPLATE = """
Ты - AI-ассистент, который помогает узнать о проектах и опыте Зарубаева Руслана.
Отвечай на вопрос пользователя, опираясь на ВОПРОС С КОНТЕКСТОМ,
ОБЩИЙ КОНТЕКСТ и на КОНТЕКСТ ПО ВОПРОСУ.
Правила ответа:
1. Если вопрос не касается проектов, опыта или компетенций Руслана,
вежливо сообщи, что можешь отвечать только на вопросы,
связанные с его портфолио;
2. Если в предоставленном контексте нет информации для ответа на вопрос,
честно сообщи об этом;
3. Не придумывай информацию.

--- ВОПРОС С КОНТЕКСТОМ ---
{question}
--- КОНЕЦ ВОПРОСА С КОНТЕКСТОМ ---

--- ОБЩИЙ КОНТЕКСТ (информация о Руслане и его проектах в портфолио) ---
{general_context}
--- КОНЕЦ ОБЩЕГО КОНТЕКСТА ---

--- КОНТЕКСТ ПО ВОПРОСУ (содержимое рекомендованных файлов) ---
{specific_content}
--- КОНЕЦ КОНТЕКСТА ПО ВОПРОСУ ---

Ответ:
"""

answer_prompt = PromptTemplate(
    template=ANSWER_PROMPT_TEMPLATE,
    input_variables=[
        "question", "general_context",
        "specific_content"]
)

Подготовим общий контекст и карту всех проектных документов.

In [34]:
# Общий контекст
general_context = "\n\n".join([doc.page_content for doc in general])
# карта проектных документов
project_map = {doc.metadata["source"]: doc.page_content for doc in projects}

Подготовим функции для всех цепочек:

In [ ]:
# Функция для цепочки суммаризатора истории переписки
def run_history_chain(prompt: PromptTemplate, chat_history: list,
                      query: str, llm_model) -> str:
  """
  Запускает цепочку суммирования истории переписки для получения ответа
  """
  history_chain = prompt | llm_model | StrOutputParser()
  return history_chain.invoke({
      "chat_history": chat_history,
      "question": query
  })

# Функция для цепочки исследователя
def run_research_chain(prompt: PromptTemplate, query: str, general_context: str,
                       project_map: dict, llm_model, retriever) -> list:
  """
  Запускает цепочку исследователя для получения списка файлов
  """
  # Подготовка переменных для шаблона
  retrieved_chunks = retriever.invoke(query)
  retrieved_chunks = "\n\n".join([doc.page_content for doc in retrieved_chunks])
  all_files = "\n".join(project_map.keys())
  # Создание цепочки
  research_chain = prompt | llm_model | JsonOutputParser()
  # Запуск цепочки
  try:
    result = research_chain.invoke({
        "general_context": general_context,
        "question": query,
        "retrieved_chunks": retrieved_chunks,
        "all_files": all_files
    })
    print(f"Найдено файлов: {len(result)}")
    return result
  except Exception as e:
    print("Ошибка парсинга ответа модели: ", e)
    return []

# Функция для цепочки ассистента
def run_answer_chain(prompt: PromptTemplate, query: str, general_context: str,
                     project_map: dict, files_to_read: list, llm_model,
                     context_length = 100000, context_ratio = 2.5) -> str:
  """
  Запускает цепочку ассистента для получения ответа
  """
  # Получение списка файлов
  if len(files_to_read) == 0:
    specific_content = "Ничего не найдено"
  else:
    # Проверка ограничения по количеству токенов
    docs = []
    total_tokens = (len(query) + len(general_context)) / context_ratio
    for file in files_to_read:
      # Проверка документа на существование в списке
      if file in project_map:
        doc_tokens = len(project_map[file]) / context_ratio
        if total_tokens + doc_tokens < context_length:
          docs.append(project_map[file])
          total_tokens += doc_tokens
        else:
          break
      else:
          print(f"Файл '{file}' не найден в project_map")
    specific_content = "\n\n".join(docs)
  # Создание цепочки
  answer_chain = prompt | llm_model | StrOutputParser()
  # Запуск цепочки
  return answer_chain.invoke({
      "general_context": general_context,
      "specific_content": specific_content,
      "question": query
  })

## Подключение интерфейса и тестирование

Функции для каждой цепочки подготовлены, объединим их в одну функцию, которая получает на вход запрос пользователя и возвращает ответ.

In [36]:
# Основная функция
def get_bot_response(message: str, history: list) -> str:
  """
  Получает на вход вопрос пользователя и историю, возвращает ответ.
  """
  # Преобразование истории в текст
  chat_history = "\n".join([
      f"**{msg['role']}:** {msg['content']}"
      for msg in history])
  # Получение вопроса с контекстом
  question = run_history_chain(
      history_prompt, chat_history, message, llm
  )

  files_to_read = run_research_chain(
      research_prompt, question, general_context,
      project_map, llm, retriever
  )

  answer = run_answer_chain(
      answer_prompt, question, general_context,
      project_map, files_to_read, llm
  )
  return answer

Проверим работу функций

In [37]:
test_query = "В каких проектах используется LightGBM"
print(get_bot_response(test_query, []))

Найдено файлов: 3
LightGBM используется в следующих проектах Зарубаева Руслана:

1.  **Стоимость автомобилей**: Проект по регрессии, где LightGBM применялся для определения рыночной стоимости автомобилей.
2.  **Заказы такси**: Проект по прогнозированию временных рядов, где LightGBM использовался для предсказания количества заказов такси.
3.  **Токсичность текста**: Проект по классификации, где LightGBM применялся для определения токсичности комментариев.


Основная функция-обработчик подготовлена, обернем ее в gradio.ChatInterface. Это автоматически создаст готовый веб-интерфейс чат-бота.

In [ ]:
# Описываем с помощью блоков для настройки
with gr.Blocks(title="AI-ассистент по GitHub-портфолио Зарубаева Руслана",
               theme=gr.themes.Default()) as demo:
    gr.Markdown(
        """
# 🤖 AI-ассистент по GitHub-портфолио Зарубаева Руслана
Задайте мне вопрос о проектах, использованных технологиях или опыте Руслана.
        """
    )

    gr.ChatInterface(
        fn=get_bot_response,
        type="messages",
        chatbot=gr.Chatbot(type="messages"),
        examples=[
            "В каких проектах использовался LightGBM?",
            "Какие инструменты использовались для деплоя модели стоимости квартир?",
            "Опиши задачу из проекта по анализу токсичности комментариев."
        ],
        textbox=gr.Textbox(
            placeholder="Задайте Ваш вопрос о проектах...",
            container=True, scale=7
        )
    )

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3c40078f8275769c41.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Найдено файлов: 3


## Итоги проекта

В рамках данного pet-проекта была успешно решена задача создания интеллектуального чат-бота, способного вести диалог и отвечать на вопросы по содержимому GitHub-репозитория с учебными проектами.

Основной целью было создание не простого RAG-бота, а продвинутого диалогового ассистента, который:
1. Понимает контекст беседы, используя историю предыдущих сообщений.
2. Анализирует большой объем информации, включая множество ipynb и md файлов.
3. Предоставляет точные ответы, основанные на содержимом файлов, а не на общих знаниях.
4. Имеет удобный веб-интерфейс для демонстрации и взаимодействия.

### ⚙️ Архитектура решения

Для достижения поставленной цели была спроектирована и реализована продвинутая трехэтапная RAG-цепочка:

1. **Этап 1:** Создание самодостаточного запроса.
  - На этом этапе специальная LLM-цепочка анализирует историю диалога и новый вопрос пользователя.
  - Генерируется "сжатый" самодостаточный запрос — либо переформулированный вопрос, либо краткое резюме диалога с добавлением нового вопроса. Это позволяет избавиться от необходимости передавать всю историю на следующие этапы.

2. **Этап 2:** Поиск релевантных файлов.
  - Самодостаточный запрос с первого этапа используется для поиска наиболее релевантных документов (чанков) в векторной базе данных ChromaDB.
  - На основе найденных чанков и общего списка файлов вторая LLM-цепочка ("исследователь") формирует список полных путей к файлам, которые необходимо прочитать для исчерпывающего ответа.

3. **Этап 3:** Генерация финального ответа.
  - Третья LLM-цепочка ("ответчик") получает на вход самодостаточный запрос, общую информацию о портфолио и полное содержимое файлов, рекомендованных "исследователем".
  - На основе этих данных генерируется финальный, развернутый ответ для пользователя с учетом всех правил (ответы только по теме, запрет на выдумывание информации).

## 🛠️ Ключевые технологии
**Оркестрация:** LangChain

**LLM:** Google Gemini 2.5 Flash

**Векторная БД:** ChromaDB

**Эмбеддинги:** sentence-transformers/all-MiniLM-L6-v2

**Интерфейс:** Gradio

## ✅ Результат
Результатом работы является полнофункциональный прототип чат-бота, протестированный в данном ноутбуке. Он успешно справляется с диалоговыми сценариями, корректно использует контекст беседы и предоставляет точные ответы на основе загруженных документов.

## 🚀 Дальнейшие шаги
Следующим шагом является упаковка разработанного решения в веб-приложение и его деплой на платформе Hugging Face Spaces для публичной демонстрации. Это позволит добавить прямую ссылку на работающего ассистента в портфолио.